In [5]:
# Dependiencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [6]:
# Prepare chromedriver
# MUST USE CHROME 79
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Mars News Title and Text

In [7]:
# Retrieve page
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [12]:
# Create soup object and parse first news article title/text
news_soup = bs(browser.html, 'html.parser')

news_title = news_soup.find("div",class_="content_title").text.strip()
news_text = news_soup.find("div",class_="rollover_description").text.strip()

print(f'''
    Title: {news_title}
    Text: {news_text}
''')


    Title: NASA's Treasure Map for Water Ice on Mars
    Text: A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.



## Mars Image

In [40]:
# Retrieve page
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [41]:
# Create soup object
img_soup = bs(browser.html, 'html.parser')

# Get popup link
img_page = 'https://www.jpl.nasa.gov' + str(img_soup.find('li', class_='slide').find('a')['data-link'])

In [72]:
# Create soup object of popup page
browser.visit(img_page)
img_large_soup = bs(browser.html, 'html.parser')

# Get large image link
img_large = 'https://www.jpl.nasa.gov' + str(img_large_soup.find('figure').find('a')['href'])

print(img_large)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23579_hires.jpg


## Mars Weather

In [74]:
# Retrieve page
weather_url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(weather_url)

In [86]:
# Crete soup object
soup = BeautifulSoup(response.text, 'html.parser')

weather = soup.find('div', class_="js-tweet-text-container").find('p').text

print(weather)

InSight sol 370 (2019-12-11) low -99.0ºC (-146.1ºF) high -19.5ºC (-3.0ºF)
winds from the SSE at 5.1 m/s (11.4 mph) gusting to 19.6 m/s (43.9 mph)
pressure at 6.60 hPapic.twitter.com/XqOy8yFlI6


## Mars Facts

In [138]:
# Retrieve page into pandas
facts_url = 'https://space-facts.com/mars/'
mars_facts_df = pd.read_html(facts_url)[0]

In [139]:
# Remove column names
mars_facts_df.columns = ['','']

mars_facts_df

,,
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


## Mars Hemispheres

In [174]:
# Retrieve pages, save title and img url to list as dictionaries
hemispheres_url_list = [
    'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
    'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
    'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
    'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

hemisphere_image_list = []

for url in range(len(hemispheres_url_list)):
    
    hemisphere_image_dict = {}
    
    response = requests.get(hemispheres_url_list[url])
    soup = BeautifulSoup(response.text, 'html.parser')
    
    title = soup.find('title').text.split('Enhanced')[0].rstrip()
    img_url = soup.find('div', class_='downloads').find('a')['href']
    
    hemisphere_image_dict['title'] = title
    hemisphere_image_dict ['img_url'] = img_url
    
    hemisphere_image_list.append(hemisphere_image_dict)
    
print(hemisphere_image_list)

[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
